# 🏢 Projet 3 : Project Lead Douglas Prédiction de la Consommation Énergétique et des Émissions CO2

## 🎯 Objectifs du Projet

Ce projet vise à construire **deux modèles de régression supervisée** pour prédire :

1. **La consommation énergétique** (`SiteEnergyUse(kBtu)`) des bâtiments non-résidentiels de Seattle
2. **Les émissions de CO2** (`TotalGHGEmissions`) associées à ces bâtiments

**Contexte :** Dataset "2016 Building Energy Benchmarking" de la ville de Seattle (3376 bâtiments initiaux).

**Méthodologie :** Analyse exploratoire → Feature Engineering → Modélisation → Optimisation → Interprétation

---

# 📊 PARTIE 0 : Analyse Exploratoire


### Import des modules


In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

### Analyse Exploratoire


In [2]:
# Charger le dataset
df = pd.read_csv("data/2016_Building_Energy_Benchmarking.csv")

In [3]:
# On regarde comment les batiments sont définis dans le dataset
df.head()

,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,Address,City,State,ZipCode,TaxParcelIdentificationNumber,...,Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),DefaultData,Comments,ComplianceStatus,Outlier,TotalGHGEmissions,GHGEmissionsIntensity
0,1,2016,NonResidential,Hotel,Mayflower park hotel,405 Olive way,Seattle,WA,98101.0,0659000030,...,1.156514e+06,3946027.0,12764.52930,1276453.0,False,NaN,Compliant,NaN,249.98,2.83
1,2,2016,NonResidential,Hotel,Paramount Hotel,724 Pine street,Seattle,WA,98101.0,0659000220,...,9.504252e+05,3242851.0,51450.81641,5145082.0,False,NaN,Compliant,NaN,295.86,2.86
2,3,2016,NonResidential,Hotel,5673-The Westin Seattle,1900 5th Avenue,Seattle,WA,98101.0,0659000475,...,1.451544e+07,49526664.0,14938.00000,1493800.0,False,NaN,Compliant,NaN,2089.28,2.19
3,5,2016,NonResidential,Hotel,HOTEL MAX,620 STEWART ST,Seattle,WA,98101.0,0659000640,...,8.115253e+05,2768924.0,18112.13086,1811213.0,False,NaN,Compliant,NaN,286.43,4.67
4,8,2016,NonResidential,Hotel,WARWICK SEATTLE HOTEL (ID8),401 LENORA ST,Seattle,WA,98121.0,0659000970,...,1.573449e+06,5368607.0,88039.98438,8803998.0,False,NaN,Compliant,NaN,505.01,2.88


In [4]:
# On regarde le nombre de valeurs manquantes par colonne ainsi que leur type
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3376 entries, 0 to 3375
Data columns (total 46 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   OSEBuildingID                    3376 non-null   int64  
 1   DataYear                         3376 non-null   int64  
 2   BuildingType                     3376 non-null   object 
 3   PrimaryPropertyType              3376 non-null   object 
 4   PropertyName                     3376 non-null   object 
 5   Address                          3376 non-null   object 
 6   City                             3376 non-null   object 
 7   State                            3376 non-null   object 
 8   ZipCode                          3360 non-null   float64
 9   TaxParcelIdentificationNumber    3376 non-null   object 
 10  CouncilDistrictCode              3376 non-null   int64  
 11  Neighborhood                     3376 non-null   object 
 12  Latitude            

In [5]:
# Affiche le pourcentage de valeurs manquantes par colonne.
print(df.isna().mean().sort_values(ascending=False))

Comments                           1.000000
Outlier                            0.990521
YearsENERGYSTARCertified           0.964751
ThirdLargestPropertyUseType        0.823460
ThirdLargestPropertyUseTypeGFA     0.823460
SecondLargestPropertyUseType       0.502666
SecondLargestPropertyUseTypeGFA    0.502666
ENERGYSTARScore                    0.249704
LargestPropertyUseTypeGFA          0.005924
LargestPropertyUseType             0.005924
ZipCode                            0.004739
ListOfAllPropertyUseTypes          0.002666
Electricity(kWh)                   0.002666
SourceEUIWN(kBtu/sf)               0.002666
GHGEmissionsIntensity              0.002666
TotalGHGEmissions                  0.002666
NaturalGas(therms)                 0.002666
SteamUse(kBtu)                     0.002666
NaturalGas(kBtu)                   0.002666
SourceEUI(kBtu/sf)                 0.002666
Electricity(kBtu)                  0.002666
NumberofBuildings                  0.002370
SiteEUI(kBtu/sf)                

#### 📖 Compréhension des colonnes principales

**Variables cibles potentielles (consommation énergétique) :**

- `SiteEnergyUse(kBtu)` : Consommation totale d'énergie sur site
- `TotalGHGEmissions` : Émissions totales de gaz à effet de serre

| Type de Feature  | Question posée         | Rôle                                                      | Impact sur la Prédiction                                                                                           |
| :--------------- | :--------------------- | :-------------------------------------------------------- | :----------------------------------------------------------------------------------------------------------------- |
| **Identitaire**  | _C'est qui ?_          | Sert à identifier, filtrer ou grouper les données.        | **Faible**<br>Le nom du bâtiment ne change pas sa physique.                                                        |
| **Structurelle** | _C'est fait comment ?_ | Caractéristiques physiques immuables (Le Contenant).      | **Physique**<br>Détermine la consommation de base (Plus c'est grand/vieux, plus ça consomme).                      |
| **Usage**        | _On y fait quoi ?_     | Le comportement et l'activité à l'intérieur (Le Contenu). | **Intensité**<br>Variable critique : explique pourquoi un Hôpital consomme 10x plus qu'un Entrepôt de même taille. |

**Features identitaires :**

- `PropertyName` : Nom du bâtiment (identification)
- `BuildingType` : Type de bâtiment (résidentiel/non-résidentiel) → **Filtre principal**
- `PrimaryPropertyType` : Catégorie détaillée (Office, School, Warehouse, etc.)

**Features structurelles :**

- `PropertyGFATotal` : Surface totale en pieds carrés (crucial pour normalisation)
- `YearBuilt` : Année de construction (âge du bâtiment)
- `NumberofBuildings` : Nombre de bâtiments dans la propriété

**Features d'usage :**

- `LargestPropertyUseType` : Usage principal du bâtiment
- `SecondLargestPropertyUseType` : Usage secondaire (si présent)
- `LargestPropertyUseTypeGFA` : Surface dédiée à l'usage principal

**Stratégie de nettoyage :**

1. Garder les colonnes avec **<50% de valeurs manquantes** ET pertinentes pour la modélisation
2. Supprimer les colonnes **constantes** (ou redondante)
3. Éliminer les lignes avec **valeurs manquantes sur les targets**
4. Détecter et retirer les **incohérences métier** (surface/énergie négatives ou nulles)


In [6]:
# Préparations des chiffres
n_total = df.shape[0]
nb_habitations = df["BuildingType"].str.contains("Multifamily").sum()

# On calcule le pourcentage dans une variable à part
pct_habitations = nb_habitations / n_total

print(f"Nombre initial de bâtiments : {n_total}")
print(
    f"   - Dont habitations (Hors périmètre) : {nb_habitations} ({pct_habitations:.1%})"  # Arrondir et faire x100 + le symbole "%"
)

# Filtre qui permet de ne garder que les bâtiments non résidentiels
df = df[~df["BuildingType"].str.contains("Multifamily")]  # (~ = NOT)
n_after_filter = df.shape[0]

print(f"Nombre de bâtiments après filtre 'Non-Res' : {n_after_filter}")


# Liste des colonnes identifiées comme inutilisables (>50% vide ou hors sujet => risque de bruit)
cols_to_drop = [
    "Comments",
    "Outlier",
    "YearsENERGYSTARCertified",
]


# Nettoyage structurel du dataset
df = df.drop(columns=cols_to_drop, errors="ignore")

# Suppression des rows où les cibles sont vides (0 vide dans notre cas donc parfait)
targets = ["SiteEnergyUse(kBtu)", "TotalGHGEmissions"]
df = df.dropna(subset=targets)

# Vérification  de la taille
print(
    f"Taille du dataset après nettoyage structurel : {df.shape}"  # (Nombre de lignes et colonnes )
)

Nombre initial de bâtiments : 3376
   - Dont habitations (Hors périmètre) : 1708 (50.6%)
Nombre de bâtiments après filtre 'Non-Res' : 1668
Taille du dataset après nettoyage structurel : (1666, 43)


In [7]:
# NETTOYAGE DES COLONNES

# Les colonnes qui n'ont qu'une seule valeur unique
const_cols = [col for col in df.columns if df[col].nunique() == 1]
print(f"Colonnes constantes supprimées : {const_cols}")
df = df.drop(columns=const_cols)

# DOUBLONS
duplicates_count = df.duplicated().sum()
print(f"Nombre de doublons trouvés : {duplicates_count}")

# Si doublons on supprime pour garder qu'une seule version unique
if duplicates_count > 0:
    df = df.drop_duplicates()
    print("Doublons supprimés.")

# INCOHÉRENCES
# Garder que les valeurs positives
nb_incoherent = len(
    df[(df["PropertyGFATotal"] <= 0) | (df["SiteEnergyUse(kBtu)"] <= 0)]
)
if nb_incoherent > 0:
    print(
        f"\nSuppression de {nb_incoherent} bâtiments incohérents (Surface ou Energie <= 0)."
    )
    df = df[df["PropertyGFATotal"] > 0]
    df = df[df["SiteEnergyUse(kBtu)"] > 0]


# GESTION DES NaN POUR VARIABLES D'USAGE
# NaN dans Third* = "pas de 3ème usage" → Information utile, pas une erreurprint(f"\n---> Dataset prêt pour la suite : {df.shape}")

df["ThirdLargestPropertyUseType"] = df["ThirdLargestPropertyUseType"].fillna("None")
df["ThirdLargestPropertyUseTypeGFA"] = df["ThirdLargestPropertyUseTypeGFA"].fillna(0)
print("   ThirdLargestPropertyUseType : NaN → 'None' (absence de 3ème usage)")
print("   ThirdLargestPropertyUseTypeGFA : NaN → 0")

Colonnes constantes supprimées : ['DataYear', 'City', 'State']
Nombre de doublons trouvés : 0

Suppression de 16 bâtiments incohérents (Surface ou Energie <= 0).
   ThirdLargestPropertyUseType : NaN → 'None' (absence de 3ème usage)
   ThirdLargestPropertyUseTypeGFA : NaN → 0


### Analyse Statistique Descriptive

| Indicateur Statistique | Définition Simple                                                 | Interprétation pour le Projet (Seattle)                                                                                                  |
| :--------------------- | :---------------------------------------------------------------- | :--------------------------------------------------------------------------------------------------------------------------------------- |
| **Moyenne** (`mean`)   | Le centre de gravité mathématique.                                | **Tendance globale**, mais fortement tirée vers le haut par les quelques bâtiments géants (Campus, Hôpitaux).                            |
| **Médiane** (`median`) | La valeur centrale (50% au-dessus, 50% en dessous).               | **Le bâtiment "Standard"**. C'est la valeur la plus représentative de la réalité de la majorité du parc immobilier.                      |
| **Écart-type** (`std`) | La dispersion des valeurs autour de la moyenne.                   | **La diversité**. Un écart-type élevé indique que le parc est très hétérogène (mélange de petits commerces et de grands buildings).      |
| **Outliers** (`3σ`)    | Valeurs dépassant le seuil statistique (Moyenne + 3x Écart-type). | **Les anomalies ou géants**. Ce sont les cas extrêmes (structures très énergivores) qu'il faudra peut-être traiter à part ou surveiller. |


In [8]:
# Statistiques sur la consommation d'énergie
print("\n🔋 CONSOMMATION D'ÉNERGIE (kBtu)")
print(f"   Moyenne : {df['SiteEnergyUse(kBtu)'].mean():,.0f} kBtu")
print(f"   Médiane : {df['SiteEnergyUse(kBtu)'].median():,.0f} kBtu")
print(f"   Écart-type : {df['SiteEnergyUse(kBtu)'].std():,.0f} kBtu")
print(f"   Min : {df['SiteEnergyUse(kBtu)'].min():,.0f} kBtu")
print(f"   Max : {df['SiteEnergyUse(kBtu)'].max():,.0f} kBtu")

# Statistiques sur les émissions CO2
print("\n🌍 ÉMISSIONS DE CO2 (Metric Tons)")
print(f"   Moyenne : {df['TotalGHGEmissions'].mean():,.0f} tons")
print(f"   Médiane : {df['TotalGHGEmissions'].median():,.0f} tons")
print(f"   Écart-type : {df['TotalGHGEmissions'].std():,.0f} tons")
print(f"   Min : {df['TotalGHGEmissions'].min():,.0f} tons")
print(f"   Max : {df['TotalGHGEmissions'].max():,.0f} tons")

# Détection des outliers
energy_mean = df["SiteEnergyUse(kBtu)"].mean()
energy_std = df["SiteEnergyUse(kBtu)"].std()
outliers_energy = df[df["SiteEnergyUse(kBtu)"] > energy_mean + 3 * energy_std]

print("\nOUTLIERS DÉTECTÉS :")
print(
    f"Bâtiments avec consommation > 3σ : {len(outliers_energy)} ({len(outliers_energy) / len(df) * 100:.1f}%)"
    # 3σ (Sigma) = 3 fois l'écart-type : c'est la frontière statistique au-delà de laquelle une valeur est considérée comme une anomalie (Outlier).
)
print(
    f"Seuil outlier : {energy_mean + 3 * energy_std:,.0f} kBtu"
)  # on détermine ce seuil en fonction de la distribution des données(moyenne et écart-type).


🔋 CONSOMMATION D'ÉNERGIE (kBtu)
   Moyenne : 8,519,756 kBtu
   Médiane : 2,593,203 kBtu
   Écart-type : 30,378,696 kBtu
   Min : 16,809 kBtu
   Max : 873,923,712 kBtu

🌍 ÉMISSIONS DE CO2 (Metric Tons)
   Moyenne : 186 tons
   Médiane : 50 tons
   Écart-type : 755 tons
   Min : -1 tons
   Max : 16,871 tons

OUTLIERS DÉTECTÉS :
Bâtiments avec consommation > 3σ : 11 (0.7%)
Seuil outlier : 99,655,845 kBtu


#### 📊 Graphiques 1 & 2 : Classement Énergétique et Écologique par Type de Bâtiment

**Objectif :** Comparer les 22 types de bâtiments pour identifier les catégories les plus consommatrices et polluantes.

**Méthodologie technique :**

- **Visualisation** : Boxplots (boîtes à moustaches) pour visualiser médiane, quartiles et outliers
- **Tri intelligent** : Classement automatique par médiane croissante (les plus économes à gauche, les plus gourmands à droite)
- **Échelle logarithmique** : Permet de comparer sur un même graphique les entrepôts (~100k kBtu) et les hôpitaux (~100M kBtu)
- **Dataset complet** : Analyse sur les 1650 bâtiments (tous types confondus)

**Graphique 1 : Classement de la Consommation d'Énergie**

- Lecture immédiate des types les plus énergivores (positionnés à droite)
- Hauteur de chaque boîte = variabilité au sein d'une catégorie
- Points isolés = outliers (bâtiments atypiques)

**Graphique 2 : Classement des Émissions de CO2**

- Même méthodologie appliquée aux émissions de gaz à effet de serre
- Permet de vérifier la corrélation entre consommation énergétique et pollution
- Confirme que les types les plus énergivores sont généralement les plus pollueurs


In [9]:
# Consommation d'Énergie
fig_energy = px.box(
    df,
    x="PrimaryPropertyType",
    y="SiteEnergyUse(kBtu)",
    color="PrimaryPropertyType",
    title="Classement de la consommation par type de bâtiment",
    log_y=True,
    points="outliers",
    hover_data=["PropertyName"],
)

# On trie l'axe X par ordre croissant de consommation (médiane)
# Permet de voir immédiatement qui sont les "mauvais élèves" à droite
fig_energy.update_xaxes(categoryorder="median ascending")

# On cache la légende qui prend trop de place
fig_energy.update_layout(showlegend=False)

fig_energy.show()

# Émissions de CO2
fig_co2 = px.box(
    df,
    x="PrimaryPropertyType",
    y="TotalGHGEmissions",
    color="PrimaryPropertyType",
    title="Classement des émissions de CO2 par type de bâtiment",
    log_y=True,
    points="outliers",
    hover_data=["PropertyName"],
)

# On trie par pollution médiane ( les plus pollueurs à droite (mauvais élèves) )
fig_co2.update_xaxes(categoryorder="median ascending")
fig_co2.update_layout(showlegend=False)

fig_co2.show()

#### 📊 Graphique 3 : Relation Surface vs Consommation (Vue Globale)

**🎯 Objectif :** Vérifier la corrélation entre la surface et la consommation sur l'ensemble du parc immobilier, et observer la relation avec les émissions CO2.

**Ce qu'on observe :**

- **Corrélation forte (Ligne Rouge) :** La ligne de tendance (Régression OLS) confirme mathématiquement que la consommation augmente avec la surface. L'hypothèse est validée.
- **Échelle Logarithmique :** Indispensable pour visualiser lisiblement les petits bâtiments (10k ft²) et les campus géants (5M+ ft²) sur le même plan.
- **Indicateur CO2 (Couleur des points) :** La couleur indique le niveau d'émissions de gaz à effet de serre. On confirme que les bâtiments les plus consommateurs sont aussi les plus pollueurs (corrélation +0.860).

**Points d'attention (Analyse des Outliers) :**

- **La "Normalité" :** La grande majorité des points se concentrent autour de la ligne rouge. C'est le comportement attendu.
- **Les Anomalies (Outliers) :**
  - **Au-dessus de la ligne rouge :** Bâtiments énergivores. Pour une même surface, ils consomment beaucoup plus que la moyenne (ex: Laboratoires, Supermarchés, ou mauvaise isolation).
  - **En-dessous de la ligne rouge :** Bâtiments économes ou à faible occupation (ex: Entrepôts, Lieux de culte).

**💡 Conclusion :** La surface est le facteur déterminant principal pour les deux targets (énergie ET CO2), mais les écarts importants autour de la ligne de tendance montrent que le _Type d'usage_ jouera un rôle clé pour affiner les prédictions.


In [10]:
fig_scatter = px.scatter(
    df,
    x="PropertyGFATotal",
    y="SiteEnergyUse(kBtu)",
    color="TotalGHGEmissions",
    opacity=0.5,
    log_x=True,
    log_y=True,
    title="Corrélation Globale : Surface vs Consommation (+ Tendance)",
    trendline="ols",  # Ajout de la ligne de régression
    trendline_color_override="red",
    hover_data=["PropertyName", "PrimaryPropertyType"],
)

fig_scatter.show()

#### 📊 Graphique 4 : Matrice de Corrélation (Heatmap)

**🎯 Objectif :** Visualiser d'un seul coup d'œil toutes les relations entre les variables numériques clés du dataset.

**Méthodologie :**

- **Variables sélectionnées** : Cibles énergétiques, caractéristiques structurelles et performances
- **Coefficient de Pearson** : Mesure la force et le sens de la relation linéaire entre deux variables (-1 à +1)
- **Lecture de la Heatmap** :
  - **Rouge foncé (proche de +1)** : Corrélation positive forte (quand l'une augmente, l'autre aussi)
  - **Bleu foncé (proche de -1)** : Corrélation négative forte (relation inverse)
  - **Blanc (proche de 0)** : Pas de corrélation linéaire

**Ce qu'on cherche :**

1. **Variables fortement corrélées aux cibles** (`SiteEnergyUse(kBtu)` ET `TotalGHGEmissions`) → Features prédictives puissantes
2. **Redondances** : Variables très corrélées entre elles → Risque de multicolinéarité (à éviter en modélisation)
3. **Relations inattendues** : Découvertes de patterns non évidents
4. **Comparaison des deux targets** : Identifier si les mêmes features prédisent l'énergie ET les émissions CO2

**💡 Utilité pour la Modélisation :**

- Sélectionner les features les plus pertinentes pour **chaque target** (Énergie ET CO2)
- Éviter d'inclure des variables redondantes qui n'apportent pas d'information supplémentaire
- Comprendre les mécanismes physiques sous-jacents (ex: Surface → Énergie → CO2)
- **Comparer les profils prédictifs** des deux targets pour adapter la stratégie de modélisation
- Valider que les deux targets ont suffisamment de corrélations avec les features disponibles


In [11]:
# Sélection des variables numériques pertinentes
# On garde les variables structurelles, énergétiques et d'usage
corr_cols = [
    "SiteEnergyUse(kBtu)",  # TARGET 1 : Consommation totale
    "TotalGHGEmissions",  # TARGET 2 : Émissions CO2
    "PropertyGFATotal",  # Surface totale (facteur principal)
    "YearBuilt",  # Âge du bâtiment (isolation, efficacité)
    "NumberofFloors",  # Hauteur (peut influencer chauffage/clim)
    "NumberofBuildings",  # Nombre de bâtiments dans le complexe
    "LargestPropertyUseTypeGFA",  # Surface du plus grand usage
    "SecondLargestPropertyUseTypeGFA",  # Surface du 2ème plus grand usage
    "ThirdLargestPropertyUseTypeGFA",  # Surface du 3ème plus grand usage
]

# Calcul de la matrice de corrélation (Pearson)
corr_matrix = df[corr_cols].corr()

fig_corr = go.Figure(
    data=go.Heatmap(
        z=corr_matrix.values,
        x=corr_matrix.columns,
        y=corr_matrix.columns,
        colorscale="RdBu_r",
        zmid=0,  # Centre de l'échelle à 0
        text=corr_matrix.values.round(2),  # Afficher les valeurs
        texttemplate="%{text}",
        textfont={"size": 10},
        colorbar=dict(title="Corrélation"),
    )
)

fig_corr.update_layout(
    title="Matrice de Corrélation des Variables Clés",
    xaxis_title="",
    yaxis_title="",
    width=900,
    height=800,
    xaxis=dict(tickangle=-45),  # Inclinaison des labels
)

fig_corr.show()


print("\n CORRÉLATIONS AVEC LES DEUX TARGETS POTENTIELLES :\n")

# Target 1 : Consommation d'énergie
print("⚡ TARGET 1 : SiteEnergyUse(kBtu) - Consommation énergétique")
target1_corr = corr_matrix["SiteEnergyUse(kBtu)"].sort_values(ascending=False)
for var, corr_value in target1_corr.items():
    if var != "SiteEnergyUse(kBtu)":  # Exclure la cible elle-même car toujours à 1.0
        print(f"   {var:30} : {corr_value:+.3f}")

# Target 2 : Émissions CO2
print("\n TARGET 2 : TotalGHGEmissions - Émissions de CO2")
target2_corr = corr_matrix["TotalGHGEmissions"].sort_values(ascending=False)
for var, corr_value in target2_corr.items():
    if var != "TotalGHGEmissions":
        print(f"   {var:30} : {corr_value:+.3f}")

print(
    "\n ANALYSE : Les deux targets sont fortement corrélées (+0.860), donc les mêmes features seront pertinentes."
)
print(
    "    Choix final : SiteEnergyUse(kBtu) et TotalGHGEmissions car 0% de valeurs manquantes."
)



 CORRÉLATIONS AVEC LES DEUX TARGETS POTENTIELLES :

⚡ TARGET 1 : SiteEnergyUse(kBtu) - Consommation énergétique
   TotalGHGEmissions              : +0.860
   LargestPropertyUseTypeGFA      : +0.846
   PropertyGFATotal               : +0.809
   NumberofBuildings              : +0.713
   SecondLargestPropertyUseTypeGFA : +0.671
   ThirdLargestPropertyUseTypeGFA : +0.330
   NumberofFloors                 : +0.219
   YearBuilt                      : +0.064

 TARGET 2 : TotalGHGEmissions - Émissions de CO2
   SiteEnergyUse(kBtu)            : +0.860
   LargestPropertyUseTypeGFA      : +0.575
   SecondLargestPropertyUseTypeGFA : +0.539
   PropertyGFATotal               : +0.528
   NumberofBuildings              : +0.418
   ThirdLargestPropertyUseTypeGFA : +0.387
   NumberofFloors                 : +0.130
   YearBuilt                      : +0.051

 ANALYSE : Les deux targets sont fortement corrélées (+0.860), donc les mêmes features seront pertinentes.
    Choix final : SiteEnergyUse(kBtu) e

### 📋 Synthèse de l'Analyse Exploratoire

#### Nettoyage et Préparation des Données

**1. Restriction au périmètre pertinent :**

- **Exclus** : 1708 bâtiments "Multifamily" (habitations hors périmètre).
- **Conservés** : 1650 bâtiments non-résidentiels (après suppression des valeurs négatives/nulles).

**2. Colonnes supprimées (arguments métier) :**

- **>50% vide** : `Comments`, `YearsENERGYSTARCertified` → Trop peu d'information exploitable.
- **Constantes** : `DataYear`, `City`, `State` → Aucun pouvoir discriminant (variance nulle).
- **Hors cible** : `Outlier` → Gestion manuelle des outliers préférée.
- **⚠️ Exception** : `ThirdLargestPropertyUseType` (50% NaN) **conservée** → NaN = "pas de 3ème usage" = information utile.

**3. Lignes supprimées :**

- **16 bâtiments** avec Surface ≤ 0 ou Énergie ≤ 0 (incohérence métier).
- **1 bâtiment** avec émissions CO2 négatives (impossible physiquement).

**4. Dataset final :**

- **1649 bâtiments** × **40 colonnes** (après nettoyage complet).
- **Targets choisies** :
  - `SiteEnergyUse(kBtu)` - Consommation énergétique
  - `TotalGHGEmissions` - Émissions de CO2
  - Raison : 0% de valeurs manquantes pour les deux, forte corrélation (+0.860) validant leur cohérence.
- **Variables d'usage conservées** : Largest, Second, Third (complexité du bâtiment → prédictif)

---

#### Insights Clés

**1. Distribution des consommations :**

- **Forte asymétrie à droite** (Moyenne >> Médiane).
- Moyenne : ~8.4M kBtu | Médiane : ~2.5M kBtu.
- **Conséquence** : Une transformation logarithmique de la target sera indispensable pour la modélisation.

**2. Influence du Type de Bâtiment (Usage) :**

- Le **PrimaryPropertyType** est un facteur discriminant majeur.
- **Gros consommateurs** : Hôpitaux, Supermarchés, Data Centers.
- **Faibles consommateurs** : Entrepôts, Lieux de culte.

**3. Analyse de Corrélation (9 variables numériques clés) :**

Nous avons écarté les variables de consommation (Élec/Gaz) pour éviter le data leakage. Voici les vrais moteurs physiques :

**Pour Target 1 (Consommation Énergétique) :**

- **`LargestPropertyUseTypeGFA` (+0.846)** : La surface dédiée à l'usage principal est le prédicteur le plus fort. C'est la combinaison surface × intensité d'usage qui explique la consommation.
- **`PropertyGFATotal` (+0.809)** : La surface totale reste un facteur majeur (relation quasi-linéaire sur échelle log).
- **`NumberofBuildings` (+0.713)** : La complexité du site (Campus, Hôpitaux multi-bâtiments) est un indicateur très fort.
- **`SecondLargestPropertyUseTypeGFA` (+0.671)** : Le 2ème usage contribue également à la diversité énergétique.
- **`ThirdLargestPropertyUseTypeGFA` (+0.330)** : La présence d'un 3ème usage distinct indique une complexité fonctionnelle (ex: hôpital avec data center).
- **`NumberofFloors` (+0.219)** : La hauteur joue un rôle secondaire.
- **`YearBuilt` (+0.064)** : Pas de corrélation linéaire directe. L'âge joue un rôle complexe (non-linéaire) qui nécessitera des modèles adaptés (arbres de décision).

**Pour Target 2 (Émissions CO2) :**

- **`SiteEnergyUse(kBtu)` (+0.860)** : Corrélation très forte avec l'énergie (logique physique : plus d'énergie = plus d'émissions).
- **`LargestPropertyUseTypeGFA` (+0.575)** : L'usage principal a un impact majeur sur les émissions (certains usages émettent plus de CO2).
- **`SecondLargestPropertyUseTypeGFA` (+0.539)** : La diversité d'usage contribue significativement aux émissions.
- **`PropertyGFATotal` (+0.528)** : La surface totale a moins d'impact direct sur les émissions que sur l'énergie (le type d'énergie utilisée compte plus).
- **`NumberofBuildings` (+0.418)** : Impact modéré de la complexité du site.
- **`ThirdLargestPropertyUseTypeGFA` (+0.387)** : La complexité fonctionnelle joue un rôle pour les émissions.
- **`NumberofFloors` (+0.130)** : Corrélation faible avec les émissions.
- **`YearBuilt` (+0.051)** : Aucune corrélation linéaire directe.

**💡 Insight clé :** Les variables de **surface par type d'usage** (Largest/Second/Third GFA) sont des prédicteurs puissants car elles capturent à la fois la taille ET l'intensité énergétique de chaque fonction du bâtiment. Un hôpital de 10 000 m² consomme bien plus qu'un entrepôt de même surface.


In [12]:
# NETTOYAGE AVANT MODÉLISATION

# Supprimer les émissions négatives (physiquement impossibles)
nb_negatif = len(df[df["TotalGHGEmissions"] < 0])
if nb_negatif > 0:
    print(f"   Suppression de {nb_negatif} bâtiment(s) avec émissions négatives")
    df = df[df["TotalGHGEmissions"] >= 0]

# Vérification
print(f" Dataset final prêt pour la modélisation : {df.shape}")
print(f"   - Bâtiments : {df.shape[0]}")
print(f"   - Variables : {df.shape[1]}")

   Suppression de 1 bâtiment(s) avec émissions négatives
 Dataset final prêt pour la modélisation : (1649, 40)
   - Bâtiments : 1649
   - Variables : 40


#### ✅ Analyse Exploratoire COMPLÈTE

**Résumé des accomplissements :**

1. **Compréhension du dataset** : Explication détaillée des colonnes clés et de leur rôle
2. **Nettoyage structuré** : Suppression justifiée de colonnes/lignes avec traçabilité complète
3. **Statistiques descriptives** : Résumé des variables cibles (énergie, CO2) avec détection d'outliers
4. **Visualisations pertinentes** :
   - Boxplots comparatifs (22 types de bâtiments classés par consommation ET émissions CO2)
   - Scatter plot avec ligne de tendance OLS (corrélation Surface vs Énergie, coloré par CO2)
   - Heatmap de corrélation (relations entre variables clés + **comparaison des 2 targets**)
5. **Choix des targets justifié** :
   - **Target 1** : `SiteEnergyUse(kBtu)` - Consommation énergétique
   - **Target 2** : `TotalGHGEmissions` - Émissions de CO2
   - Raison : 0% de valeurs manquantes, forte corrélation (+0.860), profils prédictifs différents
6. **Identification des incohérences** : Valeurs négatives/nulles détectées et supprimées

**Dataset final prêt pour la modélisation : 1649 bâtiments × 40 colonnes**
**Objectif : Construire DEUX modèles de prédiction (un par target)**

---

### 📚 Ressources Complémentaires

Pour approfondir l'esprit d'une analyse exploratoire, consultez :

https://www.kaggle.com/code/pmarcelino/comprehensive-data-exploration-with-python

_(Ce notebook n'est pas un modèle à suivre strictement, mais une source d'inspiration)_


# Modélisation


### Import des modules


In [13]:
# Selection
from sklearn.model_selection import cross_validate

# Preprocess
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Modèles
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor


### Feature Engineering


**🎯 Objectif :** Créer de nouvelles variables (features) à partir des colonnes existantes pour enrichir le pouvoir prédictif du modèle.

**⚠️ RÈGLE D'OR : Pas de Data Leakage !**

- Ne jamais utiliser des variables qui contiennent déjà la réponse (quantités d'Electricity, NaturalGas, Steam)
- Créer uniquement des features dérivées de caractéristiques **structurelles**, **d'usage** ou **géographiques**

---

**💡 Pistes de Réflexion :**

**1. Temporalité (Âge du bâtiment) :**

- `BuildingAge` : 2016 - YearBuilt
- Possibilité de créer des tranches d'âge (Ex: <10 ans, 10-30 ans, >30 ans)

**2. Structure du bâtiment :**

- `HasMultipleBuildings` : Indicateur binaire (0/1) si le site contient plusieurs bâtiments
- `FloorsPerBuilding` : Ratio NumberofFloors / NumberofBuildings (si >1 bâtiment)

**3. Multi-Usage (Proportion) :**

- `HasSecondaryUse` : Indicateur binaire si usage secondaire existe
- `SecondaryUseProportion` : SecondLargestPropertyUseTypeGFA / PropertyGFATotal (si disponible)

**4. Sources d'énergie (Présence SANS quantité) :**

- `HasElectricity` : 1 si Electricity(Kbtu) > 0, sinon 0
- `HasNaturalGas` : 1 si NaturalGas(therms) > 0, sinon 0
- `HasSteam` : 1 si SteamUse(kBtu) > 0, sinon 0
- → Ces features structurelles indiquent l'équipement disponible SANS révéler l'intensité de consommation

**5. Localisation :**

- **`DistanceToCenter`** : Distance géographique entre le bâtiment et le centre-ville de Seattle
  - Utiliser les coordonnées `Latitude` et `Longitude` du dataset
  - Centre-ville de Seattle : **Latitude 47.6062, Longitude -122.3321** (Pike Place Market)
  - Formule de distance : **Haversine** (pour calculer la distance en km entre 2 points GPS)
  - **Pourquoi c'est important ?** Les bâtiments en périphérie consomment différemment (bureaux vs zones industrielles)
- `Neighborhood` : Optionnel - Regroupement par ZipCode ou CouncilDistrictCode (haute cardinalité → créer des tranches)

---


In [14]:
from math import radians, sin, cos, sqrt, atan2

print("🛠️ Création des nouvelles features...\n")

# Feature 1 : Age du bâtiment
current_year = 2016
df["BuildingAge"] = current_year - df["YearBuilt"]
print("✅ Feature 1 : BuildingAge")
print(f"   Âge moyen : {df['BuildingAge'].mean():.1f} ans")

# Feature 2 : Indicateur site multi-bâtiments (0 ou 1)
df["HasMultipleBuildings"] = (df["NumberofBuildings"] > 1).astype(int)
print("\n✅ Feature 2 : HasMultipleBuildings")
print(
    f"   {df['HasMultipleBuildings'].sum()} sites ({df['HasMultipleBuildings'].mean():.1%})"
)

# Feature 3 : Ratio étages par bâtiment
df["FloorsPerBuilding"] = df.apply(
    lambda row: row["NumberofFloors"] / row["NumberofBuildings"]
    if row["NumberofBuildings"] > 0 and pd.notna(row["NumberofFloors"])
    else 0,
    axis=1,
)
print("\n✅ Feature 3 : FloorsPerBuilding")
print(f"   Moyenne : {df['FloorsPerBuilding'].mean():.1f} étages/bâtiment")

# Feature 4 : Indicateur usage secondaire
df["HasSecondaryUse"] = df["SecondLargestPropertyUseType"].notna().astype(int)
print("\n✅ Feature 4 : HasSecondaryUse")
print(
    f"   {df['HasSecondaryUse'].sum()} bâtiments ({df['HasSecondaryUse'].mean():.1%})"
)


# Feature 5 : Distance au centre-ville (Haversine, formule universelle de distance entre 2 points GPS sur une sphère)
def haversine(lat1, lon1, lat2, lon2):
    """Calcule la distance en km entre deux points GPS"""
    R = 6371  # Rayon de la Terre en km
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = (
        sin(dlat / 2) ** 2
        + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2) ** 2
    )
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c


center_lat, center_lon = 47.6062, -122.3321  # Pike Place Market, Seattle
df["DistanceToCenter"] = df.apply(
    lambda row: haversine(row["Latitude"], row["Longitude"], center_lat, center_lon)
    if pd.notna(row["Latitude"]) and pd.notna(row["Longitude"])
    else 0,
    axis=1,
)
print("\n✅ Feature 5 : DistanceToCenter")
print(f"   Distance moyenne : {df['DistanceToCenter'].mean():.2f} km")
print(f"   Distance max : {df['DistanceToCenter'].max():.2f} km")

# Features 6-8 : Indicateurs sources d'énergie (présence SANS quantité)

# Remplace NaN par 0 avant comparaison et conversion en int
df["HasElectricity"] = (df["Electricity(kBtu)"].fillna(0) > 0).astype(int)
df["HasNaturalGas"] = (df["NaturalGas(therms)"].fillna(0) > 0).astype(int)
df["HasSteam"] = (df["SteamUse(kBtu)"].fillna(0) > 0).astype(int)

print("\n✅ Features 6-8 : Indicateurs sources d'énergie")
print(
    f"   Électricité : {df['HasElectricity'].sum()} ({df['HasElectricity'].mean():.1%})"
)
print(
    f"   Gaz Naturel : {df['HasNaturalGas'].sum()} ({df['HasNaturalGas'].mean():.1%})"
)
print(f"   Vapeur      : {df['HasSteam'].sum()} ({df['HasSteam'].mean():.1%})")

# Feature 9 : Interaction Surface × Gaz Naturel
# Hypothèse : Les grands bâtiments avec gaz consomment/émettent différemment
df["SurfaceGasInteraction"] = df["PropertyGFATotal"] * df["HasNaturalGas"]
print("\n✅ Feature 9 : SurfaceGasInteraction")
print("   Interaction Surface × Gaz (utile pour prédire CO2)")

# Feature 10 : Ratio Énergie par Surface

# Créer une nouvelle feature "EnergyPerSurface" qui est le ratio entre la consommation d'énergie et la surface totale du bâtiment.
# Attention au risque de data leakage avec Target 1.
df["EnergyPerSurface"] = df.apply(
    lambda row: row["SiteEnergyUse(kBtu)"] / row["PropertyGFATotal"]
    if row["PropertyGFATotal"] > 0
    else 0,
    axis=1,
)
print("\n✅ Feature 10 : EnergyPerSurface")
print(f"   Ratio moyen : {df['EnergyPerSurface'].mean():.2f} kBtu/sqft")
print("   ⚠️ ATTENTION : Cette feature sera EXCLUE pour Target 1 (data leakage)")
print("   ✅ Utilisable pour Target 2 (CO2) si Target 1 exclue")

print("\n🎉 Feature Engineering terminé ! 10 nouvelles features créées")
print(f"   Taille du dataset : {df.shape}")

🛠️ Création des nouvelles features...

✅ Feature 1 : BuildingAge
   Âge moyen : 54.2 ans

✅ Feature 2 : HasMultipleBuildings
   53 sites (3.2%)

✅ Feature 3 : FloorsPerBuilding
   Moyenne : 4.0 étages/bâtiment

✅ Feature 4 : HasSecondaryUse
   854 bâtiments (51.8%)

✅ Feature 5 : DistanceToCenter
   Distance moyenne : 4.52 km
   Distance max : 14.24 km

✅ Features 6-8 : Indicateurs sources d'énergie
   Électricité : 1647 (99.9%)
   Gaz Naturel : 1185 (71.9%)
   Vapeur      : 114 (6.9%)

✅ Feature 9 : SurfaceGasInteraction
   Interaction Surface × Gaz (utile pour prédire CO2)

✅ Feature 10 : EnergyPerSurface
   Ratio moyen : 68.62 kBtu/sqft
   ⚠️ ATTENTION : Cette feature sera EXCLUE pour Target 1 (data leakage)
   ✅ Utilisable pour Target 2 (CO2) si Target 1 exclue

🎉 Feature Engineering terminé ! 10 nouvelles features créées
   Taille du dataset : (1649, 50)


**Objectif :** Préparer le dataset final pour l'entraînement des modèles de Machine Learning.

Cette section va structurer les données en :

- **X** : DataFrame des features (variables explicatives)
- **y** : Series de la target (variable à prédire)

**Organisation en 4 étapes :**


---

## 🎯 PARTIE 1 : PRÉDICTION DE LA CONSOMMATION ÉNERGÉTIQUE

**Target :** `SiteEnergyUse(kBtu)` - Consommation totale d'énergie du bâtiment

**Stratégie :** Nous allons exclure `TotalGHGEmissions` (Target 2) des features pour éviter le data leakage.

---


#### Étape 1 : Préparation des Données

**Objectif :** Préparer X et y pour la modélisation avec Pipeline sklearn

**Ce qu'on va faire :**

- Sélectionner les features (exclusion leakage + identifiants)
- Extraire la target `SiteEnergyUse(kBtu)`
- Identifier les colonnes numériques vs catégorielles
- **Utiliser Pipeline + ColumnTransformer** pour automatiser le preprocessing

**⚠️ Important :** On exclut `TotalGHGEmissions` (Target 2) ET `EnergyPerSurface` (data leakage) pour Target 1.


In [15]:
# CODE : Préparation des données pour Pipeline sklearn

print("📋 Étape 1 : Préparation X et y pour TARGET 1 (Energy)\n")

# Liste des colonnes à supprimer (data leakage + identifiants inutiles)
leakage_cols = [
    # Variables de leakage (composants directs de la target)
    "SiteEnergyUseWN(kBtu)",
    "SiteEUI(kBtu/sf)",
    "SiteEUIWN(kBtu/sf)",
    "SourceEUI(kBtu/sf)",
    "SourceEUIWN(kBtu/sf)",
    "Electricity(kWh)",
    "Electricity(kBtu)",
    "NaturalGas(therms)",
    "NaturalGas(kBtu)",
    "SteamUse(kBtu)",
    "GHGEmissionsIntensity",
]

useless_cols = [
    "OSEBuildingID",
    "PropertyName",
    "TaxParcelIdentificationNumber",
    "Address",
    "ZipCode",
    "CouncilDistrictCode",
    "Neighborhood",
    "Latitude",
    "Longitude",
    "YearBuilt",
    "ComplianceStatus",
    "DefaultData",
    "ListOfAllPropertyUseTypes",
]

special_cols = ["ENERGYSTARScore"]

cols_to_drop = leakage_cols + useless_cols + special_cols

# Suppression sécurisée
existing_cols_to_drop = [c for c in cols_to_drop if c in df.columns]
df_clean = df.drop(columns=existing_cols_to_drop)

print(f"🗑️  Colonnes supprimées : {len(existing_cols_to_drop)}")

# Extraction de la target 1
y_energy = df_clean["SiteEnergyUse(kBtu)"].copy()

# Exclusion Target 2 + EnergyPerSurface (leakage)
cols_to_exclude = ["SiteEnergyUse(kBtu)", "TotalGHGEmissions", "EnergyPerSurface"]
X_energy = df_clean.drop(
    columns=[col for col in cols_to_exclude if col in df_clean.columns]
)

# Transformation log de la target
y_energy_log = np.log1p(y_energy)

print(f"✅ X_energy : {X_energy.shape}")
print(f"✅ y_energy_log : {y_energy_log.shape}")
print("\n✅ Étape 1 terminée")

📋 Étape 1 : Préparation X et y pour TARGET 1 (Energy)

🗑️  Colonnes supprimées : 25
✅ X_energy : (1649, 22)
✅ y_energy_log : (1649,)

✅ Étape 1 terminée


#### Étape 2 : Construction du Pipeline de Preprocessing

**Pourquoi utiliser un Pipeline ?**

✅ **Évite le data leakage** : Le preprocessing (scaling, encoding) est appliqué UNIQUEMENT sur le train à chaque fold de CV  
✅ **Code propre** : Tout le preprocessing est encapsulé dans un seul objet  
✅ **Reproductible** : Facile à réutiliser et déployer

**Ce que fait notre Pipeline :**

1. **Colonnes numériques** :

   - Imputation médiane (valeurs manquantes)
   - StandardScaler (normalisation)

2. **Colonnes catégorielles** :

   - Imputation mode (valeurs manquantes)
   - OneHotEncoder (encodage)

3. **Modèle** : LinearRegression / RandomForest / SVR / LightGBM


In [ ]:
# CODE : Construction du Pipeline de Preprocessing

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

print("🔧 Étape 2 : Construction du Pipeline\n")

# Identification auto des colonnes numériques vs catégorielles
numeric_features = X_energy.select_dtypes(include=["number"]).columns.tolist()
categorical_features = X_energy.select_dtypes(exclude=["number"]).columns.tolist()

print(f"📊 Features numériques   : {len(numeric_features)}")
print(f"📊 Features catégorielles : {len(categorical_features)}")

# Pipeline pour les features numériques
""" On impute par la médiane puis on scale.
    Exemple : Si un chiffre manque, on remplace par la médiane de la colonne (la médiance est robuste aux outliers)
    
    On transforme ensuite les données pour qu'elles aient une moyenne de 0 et un écart-type de 1 (StandardScaler)
    Exemple : Si un bâtiment a 3 étages et qu'un autre en a 10, après scaling, le bâtiment à 10 étages aura une valeur plus élevée mais dans une échelle comparable aux autres features.
"""
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)

# Pipeline pour les features catégorielles
categorical_transformer = Pipeline(
    steps=[
        # Si une catégorie manque, on remplace par la valeur la plus fréquente (mode)
        ("imputer", SimpleImputer(strategy="most_frequent")),
        # Encodage OneHot : transforme chaque catégorie en une colonne binaire (0 ou 1)
        ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
    ]
)

# ColumnTransformer : applique le bon pipeline à chaque type de colonne
# Fonctionnement : Il décide quelle transformation appliquer à chaque colonne en fonction de son type (numérique ou catégorielle)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ],
    remainder="drop",  # On ne garde que les colonnes spécifiées
)

print("\n✅ Pipeline de preprocessing créé")
print("   - Imputation + Scaling pour numériques")
print("   - Imputation + OneHot pour catégorielles")

# On choisit la médiane pour boucher les trous car elle n'est pas faussée par tes "bâtiments géants" (Outliers). On parle de la médiane de la colonne concernée (calculée sur les données d'entraînement).

🔧 Étape 2 : Construction du Pipeline

📊 Features numériques   : 17
📊 Features catégorielles : 5

✅ Pipeline de preprocessing créé
   - Imputation + Scaling pour numériques
   - Imputation + OneHot pour catégorielles


#### Étape 3 : Cross-Validation avec KFold

**Pourquoi Cross-Validation plutôt que Train/Test simple ?**

❌ **Problème du Train/Test simple** :

- Le résultat dépend du `random_state` (coup de chance/malchance)
- Une seule évaluation → variance élevée
- Pas robuste

✅ **Avantages de la Cross-Validation (K-Fold)** :

- **K=5** : On découpe les données en 5 parties
- Le modèle est entraîné **5 fois** avec des validations différentes
- On moyenne les résultats → **estimation robuste**
- Chaque observation est utilisée **une fois** en validation

**💡 Bonus** : On garde `return_train_score=True` pour détecter l'overfitting


In [ ]:
from sklearn.model_selection import KFold

print("🔄 Étape 3 : Configuration Cross-Validation\n")

# KFold avec 5 splits
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Métriques à calculer
scoring = {
    "r2": "r2",
    "mae": "neg_mean_absolute_error",
    "rmse": "neg_root_mean_squared_error",
}

print("✅ Cross-Validation configurée :")
print("   - K-Fold : 5 splits")
print("   - Shuffle : True (mélange aléatoire)")
print("   - Métriques : R², MAE, RMSE")
print("\n💡 Chaque modèle sera évalué 5 fois sur des données différentes")

🔄 Étape 3 : Configuration Cross-Validation

✅ Cross-Validation configurée :
   - K-Fold : 5 splits
   - Shuffle : True (mélange aléatoire)
   - Métriques : R², MAE, RMSE

💡 Chaque modèle sera évalué 5 fois sur des données différentes


#### Étape 4 : Modélisation avec Pipeline + Cross-Validation

**Algorithmes testés :**

1. **Linear Regression** : Baseline simple
2. **Random Forest** (Bagging) : Parallélisation de 100 arbres, robuste aux outliers
3. **SVR** (RBF kernel) : Modèle non-linéaire avec noyau gaussien
4. **LightGBM** (Boosting) : Modèle gradient boosting, capte les interactions complexes

**Processus pour chaque modèle :**

1. Création du Pipeline (preprocessing + modèle)
2. Cross-validation K-Fold (5 splits)
3. Calcul des métriques moyennes (R², MAE, RMSE)
4. Détection de l'overfitting (écart Train/CV)

**⏱️ Estimation** : ~2-3 minutes (LightGBM + RF sur 5 folds)


In [18]:
# CODE : Modélisation avec Pipeline + Cross-Validation (TARGET 1)

import lightgbm as lgb
import warnings

# Supprimer les warnings sklearn sur les feature names
warnings.filterwarnings("ignore", message="X does not have valid feature names")

print("🤖 MODÉLISATION TARGET 1 (Energy) - Pipeline + Cross-Validation\n")
print("=" * 80)

# Dictionnaire des modèles à tester
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(
        n_estimators=100, random_state=42, n_jobs=-1
    ),
    "SVR": SVR(kernel="rbf", C=1.0, epsilon=0.1),
    "LightGBM": lgb.LGBMRegressor(
        n_estimators=100, random_state=42, n_jobs=-1, verbose=-1
    ),
}

# Stockage des résultats
results_energy = []

for model_name, model in models.items():
    print(f"\n🔄 Cross-Validation pour {model_name}...")

    # Construction du pipeline complet
    pipeline = Pipeline(steps=[("preprocessor", preprocessor), ("model", model)])

    # Cross-validation
    cv_results = cross_validate(
        pipeline,
        X_energy,
        y_energy_log,
        cv=cv,
        scoring=scoring,
        n_jobs=-1,
        return_train_score=True,
    )

    # Extraction des métriques (conversion en positif pour MAE/RMSE)
    r2_train_mean = cv_results["train_r2"].mean()
    r2_cv_mean = cv_results["test_r2"].mean()
    r2_cv_std = cv_results["test_r2"].std()

    mae_cv_mean = -cv_results["test_mae"].mean()
    mae_cv_std = cv_results["test_mae"].std()

    rmse_cv_mean = -cv_results["test_rmse"].mean()
    rmse_cv_std = cv_results["test_rmse"].std()

    overfit_gap = r2_train_mean - r2_cv_mean

    # Affichage
    print(f"✅ {model_name}")
    print(f"   R² Train : {r2_train_mean:.4f}")
    print(f"   R² CV    : {r2_cv_mean:.4f} ± {r2_cv_std:.4f}")
    print(f"   MAE CV   : {mae_cv_mean:.4f} ± {mae_cv_std:.4f}")
    print(f"   RMSE CV  : {rmse_cv_mean:.4f} ± {rmse_cv_std:.4f}")

    # Diagnostic overfitting
    if overfit_gap > 0.15:
        print(f"   ⚠️ Surapprentissage détecté (écart = {overfit_gap:.4f})")
    elif overfit_gap < -0.05:
        print("   ⚠️ Sous-apprentissage possible")
    else:
        print(f"   ✅ Bon équilibre (écart = {overfit_gap:.4f})")

    # Stockage
    results_energy.append(
        {
            "Modèle": model_name,
            "R² CV (mean)": r2_cv_mean,
            "R² CV (std)": r2_cv_std,
            "MAE CV": mae_cv_mean,
            "RMSE CV": rmse_cv_mean,
            "R² Train": r2_train_mean,
            "Overfit Gap": overfit_gap,
        }
    )

print("\n" + "=" * 80)
print("✅ Cross-Validation terminée pour les 4 modèles")

🤖 MODÉLISATION TARGET 1 (Energy) - Pipeline + Cross-Validation


🔄 Cross-Validation pour Linear Regression...
✅ Linear Regression
   R² Train : 0.6810
   R² CV    : -0.1041 ± 1.2968
   MAE CV   : 0.6838 ± 0.0502
   RMSE CV  : 1.2235 ± 0.6777
   ⚠️ Surapprentissage détecté (écart = 0.7850)

🔄 Cross-Validation pour Random Forest...
✅ Random Forest
   R² Train : 0.9607
   R² CV    : 0.7159 ± 0.0258
   MAE CV   : 0.5041 ± 0.0108
   RMSE CV  : 0.7027 ± 0.0428
   ⚠️ Surapprentissage détecté (écart = 0.2448)

🔄 Cross-Validation pour SVR...
✅ SVR
   R² Train : 0.7717
   R² CV    : 0.6916 ± 0.0284
   MAE CV   : 0.5279 ± 0.0151
   RMSE CV  : 0.7315 ± 0.0327
   ✅ Bon équilibre (écart = 0.0801)

🔄 Cross-Validation pour LightGBM...
✅ LightGBM
   R² Train : 0.9388
   R² CV    : 0.6944 ± 0.0151
   MAE CV   : 0.5378 ± 0.0046
   RMSE CV  : 0.7290 ± 0.0277
   ⚠️ Surapprentissage détecté (écart = 0.2444)

✅ Cross-Validation terminée pour les 4 modèles


#### 📊 Comparaison des Modèles (TARGET 1 - Energy)

**Critère de sélection** : R² CV (mean)

Le **R² CV moyen** représente la performance réelle attendue en production, car il est calculé sur des données jamais vues pendant l'entraînement.

**Lecture du tableau :**

- **R² CV (mean)** : Performance moyenne sur les 5 folds (plus élevé = meilleur)
- **R² CV (std)** : Variabilité entre les folds (plus faible = plus stable)
- **Overfit Gap** : Écart R² Train - R² CV (>0.15 = surapprentissage)


In [19]:
# CODE : Tableau comparatif et identification du champion

print("=" * 80)
print("📊 TABLEAU RÉCAPITULATIF - TARGET 1 (Energy)")
print("=" * 80 + "\n")

comparison_df_energy = pd.DataFrame(results_energy).sort_values(
    by="R² CV (mean)", ascending=False
)

print(comparison_df_energy.to_string(index=False))

# Identification du meilleur modèle
best_model_name_energy = comparison_df_energy.iloc[0]["Modèle"]
best_r2_cv_energy = comparison_df_energy.iloc[0]["R² CV (mean)"]

print("\n" + "=" * 80)
print(f"🏆 MEILLEUR MODÈLE (selon R² CV) : {best_model_name_energy}")
print(f"   R² CV = {best_r2_cv_energy:.4f}")
print("=" * 80)

print("\n✅ Comparaison terminée pour TARGET 1 (Energy)")

📊 TABLEAU RÉCAPITULATIF - TARGET 1 (Energy)

           Modèle  R² CV (mean)  R² CV (std)   MAE CV  RMSE CV  R² Train  Overfit Gap
    Random Forest      0.715889     0.025844 0.504084 0.702709  0.960673     0.244784
         LightGBM      0.694418     0.015070 0.537849 0.728994  0.938833     0.244415
              SVR      0.691559     0.028369 0.527916 0.731478  0.771653     0.080094
Linear Regression     -0.104058     1.296812 0.683769 1.223499  0.680985     0.785043

🏆 MEILLEUR MODÈLE (selon R² CV) : Random Forest
   R² CV = 0.7159

✅ Comparaison terminée pour TARGET 1 (Energy)


#### 🔧 Optimisation des Hyperparamètres - TARGET 1 (Energy)

**🎯 Objectif :** Optimiser les hyperparamètres du modèle champion (Random Forest) identifié par Cross-Validation.

**Méthode : GridSearchCV**

- **Principe** : Teste toutes les combinaisons possibles d'hyperparamètres
- **Validation** : Utilise Cross-Validation (cv=5) pour chaque combinaison
- **Avantage** : Évite le surapprentissage en validant sur données non vues
- **Combinaisons** : ~100 combinaisons (recommandation OpenClassrooms)

**💡 GridSearchCV + Pipeline + CV = Solution robuste**

- Pipeline empêche le data leakage (preprocessing uniquement sur train fold)
- CV (5 folds) donne une estimation robuste pour chaque combinaison
- Pas de `y_train_pred` : chaque combinaison est évaluée sur données non vues

**⚠️ Attention :** GridSearchCV ne garantit PAS toujours une amélioration. Les hyperparamètres par défaut peuvent déjà être optimaux pour vos données.


In [ ]:
# CODE : GridSearchCV pour Random Forest (TARGET 1)

from sklearn.model_selection import GridSearchCV

print("🔧 OPTIMISATION HYPERPARAMÈTRES - TARGET 1 (Energy)")
print("=" * 80)
print(f"Modèle à optimiser : {best_model_name_energy}")
print(f"Performance de base (R² CV) : {best_r2_cv_energy:.4f}\n")

# Grille d'hyperparamètres (limitée à ~100 combinaisons)
param_grid_energy = {
    # Le nombre d'arbres dans la forêt
    "model__n_estimators": [50, 100, 200],
    # La hauteur maximale de chaque arbre
    "model__max_depth": [10, 20, 30, None],
    # Le nombre minimum de bâtiments requis avant de créer une nouvelle branche
    "model__min_samples_split": [2, 5, 10],
    # Le nombre minimum de bâtiments qui doivent être présents dans une feuille (le résultat final)
    "model__min_samples_leaf": [1, 2],  # 2 valeurs
    # Le nombre de features à considérer lors de la recherche du meilleur split
    "model__max_features": ["sqrt", "log2"],  # 2 valeurs
}

# Calcul du nombre de combinaisons
n_combinations = 3 * 4 * 3 * 2 * 2
print(f"📊 Nombre de combinaisons à tester : {n_combinations}")
print(f"⏱️ Temps estimé : {n_combinations * 5 // 60} minutes (environ)\n")

# Construction du pipeline avec le meilleur modèle
pipeline_to_optimize = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("model", RandomForestRegressor(random_state=42, n_jobs=-1)),
    ]
)

# GridSearchCV avec Cross-Validation
print("🔄 Lancement de GridSearchCV (peut prendre plusieurs minutes)...\n")

grid_search_energy = GridSearchCV(
    estimator=pipeline_to_optimize,
    param_grid=param_grid_energy,
    cv=cv,  # Réutilise le KFold(5) défini précédemment (cross-validation)
    scoring="r2",
    n_jobs=-1,
    verbose=2,
    return_train_score=True,
)

# Entraînement (teste toutes les combinaisons)
grid_search_energy.fit(X_energy, y_energy_log)

# Résultats
print("\n" + "=" * 80)
print("✅ GridSearchCV terminé !")
print("\n🏆 MEILLEURS HYPERPARAMÈTRES TROUVÉS :")
for param, value in grid_search_energy.best_params_.items():
    print(f"   {param}: {value}")

print("\n📊 PERFORMANCES :")
print(f"   R² CV (optimisé) : {grid_search_energy.best_score_:.4f}")
print(f"   R² CV (base)     : {best_r2_cv_energy:.4f}")

# Calcul de la différence
improvement_energy = grid_search_energy.best_score_ - best_r2_cv_energy
print(
    f"\n📈 Amélioration : {improvement_energy:+.4f} ({improvement_energy / best_r2_cv_energy * 100:+.2f}%)"
)

if improvement_energy > 0:
    print("   ✅ GridSearchCV a amélioré les performances !")
else:
    print("   ⚠️ GridSearchCV n'a pas amélioré les performances.")
    print("   → Les hyperparamètres par défaut étaient déjà optimaux.")

print("=" * 80)

🔧 OPTIMISATION HYPERPARAMÈTRES - TARGET 1 (Energy)
Modèle à optimiser : Random Forest
Performance de base (R² CV) : 0.7159

📊 Nombre de combinaisons à tester : 144
⏱️ Temps estimé : 12 minutes (environ)

🔄 Lancement de GridSearchCV (peut prendre plusieurs minutes)...

Fitting 5 folds for each of 144 candidates, totalling 720 fits
[CV] END model__max_depth=10, model__max_features=sqrt, model__min_samples_leaf=1, model__min_samples_split=2, model__n_estimators=50; total time=   0.2s
[CV] END model__max_depth=10, model__max_features=sqrt, model__min_samples_leaf=1, model__min_samples_split=2, model__n_estimators=50; total time=   0.2s
[CV] END model__max_depth=10, model__max_features=sqrt, model__min_samples_leaf=1, model__min_samples_split=2, model__n_estimators=50; total time=   0.2s
[CV] END model__max_depth=10, model__max_features=sqrt, model__min_samples_leaf=1, model__min_samples_split=2, model__n_estimators=50; total time=   0.2s
[CV] END model__max_depth=10, model__max_features=sq

#### 📊 Décision Finale - TARGET 1 (Energy)

**Question :** Quel modèle utiliser pour la prédiction finale ?

**Critère de décision :** R² CV (plus élevé = meilleur)

Nous allons comparer :

1. **Modèle de base** : Random Forest avec hyperparamètres par défaut
2. **Modèle optimisé** : Random Forest avec hyperparamètres optimisés par GridSearchCV

Le modèle avec le **meilleur R² CV** sera retenu pour :

- L'analyse de Feature Importance
- Les prédictions futures
- La conclusion du projet


In [21]:
# CODE : Comparaison Modèle de Base vs Modèle Optimisé (TARGET 1)

print("=" * 80)
print("📊 COMPARAISON FINALE - TARGET 1 (Energy)")
print("=" * 80 + "\n")

# Création du DataFrame de comparaison
comparison_final_energy = pd.DataFrame(
    {
        "Modèle": ["Random Forest (Base)", "Random Forest (Optimisé)"],
        "R² CV": [best_r2_cv_energy, grid_search_energy.best_score_],
        "Hyperparamètres": ["Par défaut", "GridSearchCV"],
    }
)

# Tri par R² CV décroissant
comparison_final_energy = comparison_final_energy.sort_values(
    "R² CV", ascending=False
).reset_index(drop=True)

print(comparison_final_energy.to_string(index=False))
print("\n" + "=" * 80)

# Identification du meilleur modèle FINAL
if best_r2_cv_energy >= grid_search_energy.best_score_:
    final_model_name_energy = "Random Forest (Base)"
    final_r2_energy = best_r2_cv_energy
    # Reconstruire le pipeline de base
    final_pipeline_energy = Pipeline(
        [
            ("preprocessor", preprocessor),
            (
                "model",
                RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
            ),
        ]
    )
    final_pipeline_energy.fit(X_energy, y_energy_log)
    print("🏆 MODÈLE RETENU : Random Forest avec hyperparamètres PAR DÉFAUT")
    print(f"   R² CV = {final_r2_energy:.4f}")
    print(
        "\n💡 Conclusion : Les hyperparamètres par défaut sont optimaux pour ce dataset."
    )
else:
    final_model_name_energy = "Random Forest (Optimisé)"
    final_r2_energy = grid_search_energy.best_score_
    final_pipeline_energy = grid_search_energy.best_estimator_
    print("🏆 MODÈLE RETENU : Random Forest avec hyperparamètres OPTIMISÉS")
    print(f"   R² CV = {final_r2_energy:.4f}")
    print("\n💡 Conclusion : GridSearchCV a amélioré les performances.")

print("=" * 80)

📊 COMPARAISON FINALE - TARGET 1 (Energy)

                  Modèle    R² CV Hyperparamètres
Random Forest (Optimisé) 0.728830    GridSearchCV
    Random Forest (Base) 0.715889      Par défaut

🏆 MODÈLE RETENU : Random Forest avec hyperparamètres OPTIMISÉS
   R² CV = 0.7288

💡 Conclusion : GridSearchCV a amélioré les performances.


#### Feature Importance - TARGET 1 (Energy)

**Objectif** : Identifier les variables les plus importantes pour prédire la consommation énergétique.

**Méthode** :

1. Entraîner le meilleur modèle (identifié par CV) sur **TOUT** le dataset
2. Extraire les feature importances du modèle
3. Visualiser les TOP 15 features

**💡 Pourquoi entraîner sur tout le dataset ?**

- La CV nous a donné une estimation robuste de la performance
- Maintenant on veut le meilleur modèle possible pour l'interprétation
- Utiliser toutes les données maximise l'apprentissage


In [22]:
# CODE : Feature Importance pour le meilleur modèle (TARGET 1)

print("📊 Feature Importance - TARGET 1 (Energy)\n")
print("=" * 80)

# Utilisation du modèle FINAL (base ou optimisé selon comparaison)
print(f"🔄 Entraînement de {final_model_name_energy} sur tout le dataset...")

# On réentraîne le pipeline final sur tout le dataset
final_pipeline_energy.fit(X_energy, y_energy_log)
print("✅ Modèle entraîné\n")

# Extraction du modèle du pipeline
final_model_energy = final_pipeline_energy.named_steps["model"]

# Extraction des feature importances
if hasattr(final_model_energy, "feature_importances_"):
    # Pour RandomForest / LightGBM
    feature_importances_energy = final_model_energy.feature_importances_

    # Récupération des noms de features après preprocessing
    # NumericFeatures gardent leur nom, CategoricalFeatures sont encodées
    feature_names_energy = (
        numeric_features
        + final_pipeline_energy.named_steps["preprocessor"]
        .named_transformers_["cat"]
        .named_steps["onehot"]
        .get_feature_names_out(categorical_features)
        .tolist()
    )

    # Création du DataFrame
    importance_df_energy = pd.DataFrame(
        {"Feature": feature_names_energy, "Importance": feature_importances_energy}
    ).sort_values(by="Importance", ascending=False)

    # Calcul des pourcentages
    importance_df_energy["Importance (%)"] = (
        importance_df_energy["Importance"]
        / importance_df_energy["Importance"].sum()
        * 100
    )

    # Affichage TOP 15
    top_15_energy = importance_df_energy.head(15)

    print("🔝 TOP 15 Features les plus importantes :\n")
    for idx, row in top_15_energy.iterrows():
        print(f"{row['Feature']:40} : {row['Importance (%)']:5.2f}%")

    # Calcul du TOP 3
    top_3_energy = importance_df_energy.head(3)
    top_3_pct_energy = top_3_energy["Importance (%)"].sum()

    print(
        f"\n💡 Les 3 features les plus importantes représentent {top_3_pct_energy:.2f}% de l'importance totale"
    )

else:
    print(f"⚠️ {final_model_name_energy} ne supporte pas feature_importances_")
    print("   (Normal pour Linear Regression et SVR)")

print("\n" + "=" * 80)
print("✅ Analyse Feature Importance terminée")

📊 Feature Importance - TARGET 1 (Energy)

🔄 Entraînement de Random Forest (Optimisé) sur tout le dataset...
✅ Modèle entraîné

🔝 TOP 15 Features les plus importantes :

PropertyGFABuilding(s)                   : 16.78%
PropertyGFATotal                         : 13.71%
LargestPropertyUseTypeGFA                :  9.31%
SurfaceGasInteraction                    :  9.05%
NumberofFloors                           :  5.94%
SecondLargestPropertyUseTypeGFA          :  4.79%
FloorsPerBuilding                        :  3.68%
BuildingAge                              :  3.26%
PropertyGFAParking                       :  2.67%
DistanceToCenter                         :  2.66%
ThirdLargestPropertyUseTypeGFA           :  1.81%
PrimaryPropertyType_Warehouse            :  1.73%
PrimaryPropertyType_Supermarket / Grocery Store :  1.52%
PrimaryPropertyType_Large Office         :  1.42%
HasNaturalGas                            :  1.34%

💡 Les 3 features les plus importantes représentent 39.80% de l'importance

---

## 🎯 PARTIE 2 : PRÉDICTION DES ÉMISSIONS DE CO2

**Target :** `TotalGHGEmissions` - Émissions totales de gaz à effet de serre

**Différence avec Target 1 :**
- On peut maintenant utiliser `SiteEnergyUse(kBtu)` ET `EnergyPerSurface` comme features
- Ces variables sont exclues du leakage car elles ne sont PAS calculées à partir de CO2

**Approche :** Même méthodologie que Target 1 (Pipeline + Cross-Validation)

---


#### Étape 1 : Préparation des Données (TARGET 2 - CO2)

**Différence majeure avec Target 1** :

- ✅ `SiteEnergyUse(kBtu)` est MAINTENANT une feature (pas du leakage)
- ✅ `EnergyPerSurface` est MAINTENANT une feature (ratio créé à partir de Energy, pas de CO2)

**Ce qu'on exclut** :

- ❌ `TotalGHGEmissions` (c'est notre target)
- ❌ Variables de leakage CO2 (GHGEmissionsIntensity, etc.)


#### Étape 2 : Pipeline + Cross-Validation (TARGET 2)

**Même approche que Target 1** :

1. Construction du preprocessor (même que Target 1)
2. Configuration K-Fold (K=5)
3. Test des 4 algorithmes avec cross_validate
4. Identification du champion

**💡 Différence** : `EnergyPerSurface` et `SiteEnergyUse(kBtu)` sont maintenant dans X_co2


In [23]:
# CODE : Préparation et Modélisation TARGET 2 (CO2) avec Pipeline + CV

# Supprimer les warnings sklearn sur les feature names
warnings.filterwarnings("ignore", message="X does not have valid feature names")

print("=" * 80)
print("🌍 TARGET 2 : PRÉDICTION DES ÉMISSIONS DE CO2")
print("=" * 80 + "\n")

# Préparation des données
print("📋 Étape 1 : Préparation X_co2 et y_co2\n")

# Pour CO2, on peut utiliser SiteEnergyUse(kBtu) et EnergyPerSurface
y_co2 = df_clean["TotalGHGEmissions"].copy()

# Exclusion seulement de la target CO2
X_co2 = df_clean.drop(columns=["TotalGHGEmissions"])

# Transformation log
y_co2_log = np.log1p(y_co2)

print(f"✅ X_co2 : {X_co2.shape}")
print(f"✅ y_co2_log : {y_co2_log.shape}")
print("💡 X_co2 INCLUT maintenant SiteEnergyUse(kBtu) et EnergyPerSurface\n")

# Identification des colonnes numériques vs catégorielles
numeric_features_co2 = X_co2.select_dtypes(include=["number"]).columns.tolist()
categorical_features_co2 = X_co2.select_dtypes(exclude=["number"]).columns.tolist()

print(f"📊 Features numériques   : {len(numeric_features_co2)}")
print(f"📊 Features catégorielles : {len(categorical_features_co2)}\n")

# Construction du preprocessor pour CO2
numeric_transformer_co2 = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)

categorical_transformer_co2 = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
    ]
)

preprocessor_co2 = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer_co2, numeric_features_co2),
        ("cat", categorical_transformer_co2, categorical_features_co2),
    ],
    remainder="drop",
)

print("✅ Pipeline de preprocessing CO2 créé\n")

# Cross-Validation pour les 4 modèles
print("🔄 Étape 2 : Cross-Validation avec K-Fold (5 splits)\n")
print("=" * 80)

# Recréation du dictionnaire de modèles pour éviter les warnings
models_co2 = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(
        n_estimators=100, random_state=42, n_jobs=-1
    ),
    "SVR": SVR(kernel="rbf", C=1.0, epsilon=0.1),
    "LightGBM": lgb.LGBMRegressor(
        n_estimators=100, random_state=42, n_jobs=-1, verbose=-1
    ),
}

results_co2 = []

for model_name, model in models_co2.items():
    print(f"\n🔄 Cross-Validation pour {model_name}...")

    pipeline_co2 = Pipeline(
        steps=[("preprocessor", preprocessor_co2), ("model", model)]
    )

    cv_results_co2 = cross_validate(
        pipeline_co2,
        X_co2,
        y_co2_log,
        cv=cv,
        scoring=scoring,
        n_jobs=-1,
        return_train_score=True,
    )

    r2_train_mean_co2 = cv_results_co2["train_r2"].mean()
    r2_cv_mean_co2 = cv_results_co2["test_r2"].mean()
    r2_cv_std_co2 = cv_results_co2["test_r2"].std()

    mae_cv_mean_co2 = -cv_results_co2["test_mae"].mean()
    rmse_cv_mean_co2 = -cv_results_co2["test_rmse"].mean()

    overfit_gap_co2 = r2_train_mean_co2 - r2_cv_mean_co2

    print(f"✅ {model_name}")
    print(f"   R² Train : {r2_train_mean_co2:.4f}")
    print(f"   R² CV    : {r2_cv_mean_co2:.4f} ± {r2_cv_std_co2:.4f}")
    print(f"   MAE CV   : {mae_cv_mean_co2:.4f}")
    print(f"   RMSE CV  : {rmse_cv_mean_co2:.4f}")

    if overfit_gap_co2 > 0.15:
        print(f"   ⚠️ Surapprentissage (écart = {overfit_gap_co2:.4f})")
    elif overfit_gap_co2 < -0.05:
        print("   ⚠️ Sous-apprentissage")
    else:
        print(f"   ✅ Bon équilibre (écart = {overfit_gap_co2:.4f})")

    results_co2.append(
        {
            "Modèle": model_name,
            "R² CV (mean)": r2_cv_mean_co2,
            "R² CV (std)": r2_cv_std_co2,
            "MAE CV": mae_cv_mean_co2,
            "RMSE CV": rmse_cv_mean_co2,
            "R² Train": r2_train_mean_co2,
            "Overfit Gap": overfit_gap_co2,
        }
    )

print("✅ Cross-Validation terminée pour TARGET 2 (CO2)")
print("\n" + "=" * 80)

🌍 TARGET 2 : PRÉDICTION DES ÉMISSIONS DE CO2

📋 Étape 1 : Préparation X_co2 et y_co2

✅ X_co2 : (1649, 24)
✅ y_co2_log : (1649,)
💡 X_co2 INCLUT maintenant SiteEnergyUse(kBtu) et EnergyPerSurface

📊 Features numériques   : 19
📊 Features catégorielles : 5

✅ Pipeline de preprocessing CO2 créé

🔄 Étape 2 : Cross-Validation avec K-Fold (5 splits)


🔄 Cross-Validation pour Linear Regression...
✅ Linear Regression
   R² Train : 0.7885
   R² CV    : 0.0133 ± 1.2148
   MAE CV   : 0.6594
   RMSE CV  : 1.2610
   ⚠️ Surapprentissage (écart = 0.7752)

🔄 Cross-Validation pour Random Forest...
✅ Random Forest
   R² Train : 0.9886
   R² CV    : 0.9184 ± 0.0164
   MAE CV   : 0.2754
   RMSE CV  : 0.4118
   ✅ Bon équilibre (écart = 0.0702)

🔄 Cross-Validation pour SVR...
✅ SVR
   R² Train : 0.9051
   R² CV    : 0.8495 ± 0.0345
   MAE CV   : 0.3872
   RMSE CV  : 0.5582
   ✅ Bon équilibre (écart = 0.0555)

🔄 Cross-Validation pour LightGBM...
✅ LightGBM
   R² Train : 0.9846
   R² CV    : 0.9214 ± 0.0157
  

#### 📊 Comparaison des Modèles (TARGET 2 - CO2)


In [24]:
# CODE : Tableau comparatif TARGET 2 (CO2)

print("=" * 80)
print("📊 TABLEAU RÉCAPITULATIF - TARGET 2 (CO2)")
print("=" * 80 + "\n")

comparison_df_co2 = pd.DataFrame(results_co2).sort_values(
    by="R² CV (mean)", ascending=False
)

print(comparison_df_co2.to_string(index=False))

best_model_name_co2 = comparison_df_co2.iloc[0]["Modèle"]
best_r2_cv_co2 = comparison_df_co2.iloc[0]["R² CV (mean)"]

print("\n" + "=" * 80)
print(f"🏆 MEILLEUR MODÈLE (selon R² CV) : {best_model_name_co2}")
print(f"   R² CV = {best_r2_cv_co2:.4f}")
print("=" * 80)

print("\n✅ Comparaison terminée pour TARGET 2 (CO2)")

📊 TABLEAU RÉCAPITULATIF - TARGET 2 (CO2)

           Modèle  R² CV (mean)  R² CV (std)   MAE CV  RMSE CV  R² Train  Overfit Gap
         LightGBM      0.921426     0.015745 0.282859 0.404151  0.984644     0.063219
    Random Forest      0.918396     0.016388 0.275439 0.411800  0.988643     0.070247
              SVR      0.849547     0.034475 0.387164 0.558243  0.905087     0.055539
Linear Regression      0.013299     1.214777 0.659375 1.261041  0.788465     0.775166

🏆 MEILLEUR MODÈLE (selon R² CV) : LightGBM
   R² CV = 0.9214

✅ Comparaison terminée pour TARGET 2 (CO2)


#### 🔧 Optimisation des Hyperparamètres - TARGET 2 (CO2)

**🎯 Objectif :** Optimiser les hyperparamètres du modèle champion (LightGBM) identifié par Cross-Validation.

**Grille adaptée à LightGBM :**

- `n_estimators` : Nombre d'arbres
- `max_depth` : Profondeur maximale des arbres
- `learning_rate` : Taux d'apprentissage (vitesse de convergence)
- `num_leaves` : Nombre de feuilles par arbre (spécifique LightGBM)
- `min_child_samples` : Nombre minimal d'observations par feuille

**💡 LightGBM vs Random Forest :**

LightGBM utilise le **boosting** (arbres séquentiels qui corrigent les erreurs des précédents), contrairement à Random Forest qui utilise le **bagging** (arbres indépendants en parallèle). Les hyperparamètres sont donc différents.


In [ ]:
# CODE : GridSearchCV pour LightGBM (TARGET 2)

print("🔧 OPTIMISATION HYPERPARAMÈTRES - TARGET 2 (CO2)")
print("=" * 80)
print(f"Modèle à optimiser : {best_model_name_co2}")
print(f"Performance de base (R² CV) : {best_r2_cv_co2:.4f}\n")

# Grille d'hyperparamètres pour LightGBM (limitée à ~100 combinaisons)
param_grid_co2 = {
    "model__n_estimators": [50, 100, 200],
    "model__max_depth": [10, 20, 30],
    "model__learning_rate": [0.01, 0.1, 0.2],
    "model__num_leaves": [31, 50],
    "model__min_child_samples": [20, 50],
}

# Calcul du nombre de combinaisons
n_combinations_co2 = 3 * 3 * 3 * 2 * 2
print(f"📊 Nombre de combinaisons à tester : {n_combinations_co2}")
print(f"⏱️ Temps estimé : {n_combinations_co2 * 5 // 60} minutes (environ)\n")

# Construction du pipeline avec LightGBM
pipeline_to_optimize_co2 = Pipeline(
    [
        ("preprocessor", preprocessor_co2),
        ("model", lgb.LGBMRegressor(random_state=42, n_jobs=-1, verbose=-1)),
    ]
)

# GridSearchCV avec Cross-Validation
print("🔄 Lancement de GridSearchCV (peut prendre plusieurs minutes)...\n")

grid_search_co2 = GridSearchCV(
    estimator=pipeline_to_optimize_co2,
    param_grid=param_grid_co2,
    cv=cv,  # Réutilise le KFold(5) défini précédemment
    scoring="r2",
    n_jobs=-1,
    verbose=2,
    return_train_score=True,
)

# Entraînement (teste toutes les combinaisons)
grid_search_co2.fit(X_co2, y_co2_log)

# Résultats
print("\n" + "=" * 80)
print("✅ GridSearchCV terminé !")
print("\n🏆 MEILLEURS HYPERPARAMÈTRES TROUVÉS :")
for param, value in grid_search_co2.best_params_.items():
    print(f"   {param}: {value}")

print("\n📊 PERFORMANCES :")
print(f"   R² CV (optimisé) : {grid_search_co2.best_score_:.4f}")
print(f"   R² CV (base)     : {best_r2_cv_co2:.4f}")

# Calcul de la différence
improvement_co2 = grid_search_co2.best_score_ - best_r2_cv_co2
print(
    f"\n📈 Amélioration : {improvement_co2:+.4f} ({improvement_co2 / best_r2_cv_co2 * 100:+.2f}%)"
)

if improvement_co2 > 0:
    print("   ✅ GridSearchCV a amélioré les performances !")
else:
    print("   ⚠️ GridSearchCV n'a pas amélioré les performances.")
    print("   → Les hyperparamètres par défaut étaient déjà optimaux.")

print("=" * 80)

🔧 OPTIMISATION HYPERPARAMÈTRES - TARGET 2 (CO2)
Modèle à optimiser : LightGBM
Performance de base (R² CV) : 0.9214

📊 Nombre de combinaisons à tester : 108
⏱️ Temps estimé : 9 minutes (environ)

🔄 Lancement de GridSearchCV (peut prendre plusieurs minutes)...

Fitting 5 folds for each of 108 candidates, totalling 540 fits
[CV] END model__learning_rate=0.01, model__max_depth=10, model__min_child_samples=20, model__n_estimators=50, model__num_leaves=31; total time=   0.8s
[CV] END model__learning_rate=0.01, model__max_depth=10, model__min_child_samples=20, model__n_estimators=50, model__num_leaves=31; total time=   0.7s
[CV] END model__learning_rate=0.01, model__max_depth=10, model__min_child_samples=20, model__n_estimators=50, model__num_leaves=31; total time=   0.9s
[CV] END model__learning_rate=0.01, model__max_depth=10, model__min_child_samples=20, model__n_estimators=50, model__num_leaves=31; total time=   0.9s
[CV] END model__learning_rate=0.01, model__max_depth=10, model__min_child

#### 📊 Décision Finale - TARGET 2 (CO2)

**Question :** Quel modèle utiliser pour la prédiction finale ?

Nous allons comparer :

1. **Modèle de base** : LightGBM avec hyperparamètres par défaut
2. **Modèle optimisé** : LightGBM avec hyperparamètres optimisés par GridSearchCV

Le modèle avec le **meilleur R² CV** sera retenu.


In [26]:
# CODE : Comparaison Modèle de Base vs Modèle Optimisé (TARGET 2)

print("=" * 80)
print("📊 COMPARAISON FINALE - TARGET 2 (CO2)")
print("=" * 80 + "\n")

# Création du DataFrame de comparaison
comparison_final_co2 = pd.DataFrame(
    {
        "Modèle": ["LightGBM (Base)", "LightGBM (Optimisé)"],
        "R² CV": [best_r2_cv_co2, grid_search_co2.best_score_],
        "Hyperparamètres": ["Par défaut", "GridSearchCV"],
    }
)

# Tri par R² CV décroissant
comparison_final_co2 = comparison_final_co2.sort_values(
    "R² CV", ascending=False
).reset_index(drop=True)

print(comparison_final_co2.to_string(index=False))
print("\n" + "=" * 80)

# Identification du meilleur modèle FINAL
if best_r2_cv_co2 >= grid_search_co2.best_score_:
    final_model_name_co2 = "LightGBM (Base)"
    final_r2_co2 = best_r2_cv_co2
    # Reconstruire le pipeline de base
    final_pipeline_co2 = Pipeline(
        [
            ("preprocessor", preprocessor_co2),
            (
                "model",
                lgb.LGBMRegressor(
                    n_estimators=100, random_state=42, n_jobs=-1, verbose=-1
                ),
            ),
        ]
    )
    final_pipeline_co2.fit(X_co2, y_co2_log)
    print("🏆 MODÈLE RETENU : LightGBM avec hyperparamètres PAR DÉFAUT")
    print(f"   R² CV = {final_r2_co2:.4f}")
    print(
        "\n💡 Conclusion : Les hyperparamètres par défaut sont optimaux pour ce dataset."
    )
else:
    final_model_name_co2 = "LightGBM (Optimisé)"
    final_r2_co2 = grid_search_co2.best_score_
    final_pipeline_co2 = grid_search_co2.best_estimator_
    print("🏆 MODÈLE RETENU : LightGBM avec hyperparamètres OPTIMISÉS")
    print(f"   R² CV = {final_r2_co2:.4f}")
    print("\n💡 Conclusion : GridSearchCV a amélioré les performances.")

print("=" * 80)

📊 COMPARAISON FINALE - TARGET 2 (CO2)

             Modèle    R² CV Hyperparamètres
LightGBM (Optimisé) 0.921848    GridSearchCV
    LightGBM (Base) 0.921426      Par défaut

🏆 MODÈLE RETENU : LightGBM avec hyperparamètres OPTIMISÉS
   R² CV = 0.9218

💡 Conclusion : GridSearchCV a amélioré les performances.


#### Feature Importance - TARGET 2 (CO2)


In [27]:
# CODE : Feature Importance TARGET 2 (CO2)

print("📊 Feature Importance - TARGET 2 (CO2)\n")
print("=" * 80)

# Utilisation du modèle FINAL (base ou optimisé selon comparaison)
print(f"🔄 Entraînement de {final_model_name_co2} sur tout le dataset...")

# On réentraîne le pipeline final sur tout le dataset (si pas déjà fait)
if final_model_name_co2 == "LightGBM (Optimisé)":
    # Le pipeline est déjà entraîné depuis grid_search_co2.best_estimator_
    pass
else:
    # On réentraîne le pipeline de base
    final_pipeline_co2.fit(X_co2, y_co2_log)

print("✅ Modèle entraîné\n")

# Extraction du modèle du pipeline
final_model_co2 = final_pipeline_co2.named_steps["model"]

if hasattr(final_model_co2, "feature_importances_"):
    feature_importances_co2 = final_model_co2.feature_importances_

    feature_names_co2 = (
        numeric_features_co2
        + final_pipeline_co2.named_steps["preprocessor"]
        .named_transformers_["cat"]
        .named_steps["onehot"]
        .get_feature_names_out(categorical_features_co2)
        .tolist()
    )

    importance_df_co2 = pd.DataFrame(
        {"Feature": feature_names_co2, "Importance": feature_importances_co2}
    ).sort_values(by="Importance", ascending=False)

    importance_df_co2["Importance (%)"] = (
        importance_df_co2["Importance"] / importance_df_co2["Importance"].sum() * 100
    )

    top_15_co2 = importance_df_co2.head(15)

    print("🔝 TOP 15 Features les plus importantes :\n")
    for idx, row in top_15_co2.iterrows():
        print(f"{row['Feature']:40} : {row['Importance (%)']:5.2f}%")

    top_3_co2 = importance_df_co2.head(3)
    top_3_pct_co2 = top_3_co2["Importance (%)"].sum()

    print(
        f"\n💡 Les 3 features les plus importantes représentent {top_3_pct_co2:.2f}% de l'importance totale"
    )

else:
    print(f"⚠️ {best_model_name_co2} ne supporte pas feature_importances_")

print("\n" + "=" * 80)
print("✅ Analyse Feature Importance terminée")

📊 Feature Importance - TARGET 2 (CO2)

🔄 Entraînement de LightGBM (Optimisé) sur tout le dataset...
✅ Modèle entraîné

🔝 TOP 15 Features les plus importantes :

SiteEnergyUse(kBtu)                      : 16.97%
EnergyPerSurface                         : 12.05%
DistanceToCenter                         : 10.15%
BuildingAge                              :  9.52%
LargestPropertyUseTypeGFA                :  6.78%
SurfaceGasInteraction                    :  5.79%
PropertyGFABuilding(s)                   :  3.77%
PropertyGFATotal                         :  3.49%
SecondLargestPropertyUseTypeGFA          :  3.41%
HasNaturalGas                            :  3.33%
ThirdLargestPropertyUseTypeGFA           :  3.13%
HasSteam                                 :  2.58%
PropertyGFAParking                       :  2.42%
NumberofFloors                           :  2.10%
LargestPropertyUseType_Office            :  1.86%

💡 Les 3 features les plus importantes représentent 39.18% de l'importance totale

✅ Anal

---

# 🏁 CONCLUSION FINALE DU PROJET

## 📊 Récapitulatif des Deux Targets

**Ce projet a permis de construire deux modèles de régression supervisée performants pour prédire :**
1. **La consommation énergétique** (`SiteEnergyUse(kBtu)`)
2. **Les émissions de CO2** (`TotalGHGEmissions`)

---

### ✅ Résultats Target 1 - Consommation Énergétique

**Meilleur modèle :** Random Forest **OPTIMISÉ** (GridSearchCV)

**Performances obtenues :**

- **R² CV (optimisé) : 0.7288** (73% de variance expliquée)
- **R² CV (base) : 0.7159**
- **Amélioration GridSearchCV : +1.8%** 🚀
- MAE CV : 0.5041
- RMSE CV : 0.7027

**🔧 Optimisation par GridSearchCV :**

J'ai appliqué GridSearchCV avec Cross-Validation (K=5) pour optimiser les hyperparamètres du modèle champion Random Forest. GridSearchCV a testé **144 combinaisons** d'hyperparamètres (n_estimators, max_depth, min_samples_split, min_samples_leaf, max_features). Les résultats montrent une amélioration de **+1.8%**, confirmant que l'optimisation apporte un gain de performance mesurable.

**💡 Pourquoi GridSearchCV fonctionne avec Pipeline + CV ?**

Contrairement à l'approche Train/Test unique, l'utilisation de GridSearchCV avec Pipeline + Cross-Validation permet une optimisation robuste qui améliore réellement les performances, car :

- Chaque combinaison est évaluée sur **5 folds différents**
- Le Pipeline empêche le data leakage (preprocessing uniquement sur train fold)
- Pas de suroptimisation sur un seul split

**Variables les plus importantes** (Feature Importance du modèle optimisé) :

1. `PropertyGFABuilding(s)` - Surface des bâtiments **(16.78%)**
2. `PropertyGFATotal` - Surface totale de la propriété **(13.71%)**
3. `LargestPropertyUseTypeGFA` - Surface du type d'usage dominant **(9.31%)**
4. `SurfaceGasInteraction` - Feature d'interaction créée (Surface × Gaz naturel) **(9.05%)**

💡 Les 3 features les plus importantes représentent **39.80%** de l'importance totale

---

### ✅ Résultats Target 2 - Émissions de CO2

**Meilleur modèle :** LightGBM **OPTIMISÉ** (GridSearchCV)

**Performances obtenues :**

- **R² CV (optimisé) : 0.9218** (92% de variance expliquée) 🎯 **Performance exceptionnelle !**
- **R² CV (base) : 0.9214**
- **Amélioration GridSearchCV : +0.05%** ✅
- MAE CV : 0.2829
- RMSE CV : 0.4042

**🔧 Optimisation par GridSearchCV :**

J'ai appliqué GridSearchCV pour optimiser les hyperparamètres de LightGBM. GridSearchCV a testé **108 combinaisons** d'hyperparamètres spécifiques au boosting (n_estimators, max_depth, learning_rate, num_leaves, min_child_samples). Bien que l'amélioration soit légère (+0.05%), elle confirme que les hyperparamètres optimisés sont adaptés au dataset.

**Variables les plus importantes** :

1. **`SiteEnergyUse(kBtu)`** - Consommation énergétique totale **(16.97%)**
2. **`EnergyPerSurface`** - Intensité énergétique (kBtu/ft²) ← Feature créée ! **(12.05%)**
3. **`DistanceToCenter`** - Distance au centre de Seattle **(10.15%)**
4. `BuildingAge` - Âge du bâtiment **(9.52%)**
5. `LargestPropertyUseTypeGFA` - Surface du type d'usage dominant **(6.78%)**
6. **`SurfaceGasInteraction`** - Interaction Surface × Gaz naturel ← Feature créée ! **(5.79%)**

💡 Les 3 features les plus importantes représentent **39.18%** de l'importance totale

**💡 Insight majeur :** `SiteEnergyUse` domine (#1 avec 17%), confirmant que la consommation énergétique est le prédicteur principal des émissions CO2. Les features créées (`EnergyPerSurface` #2 et `SurfaceGasInteraction` #6) restent très importantes.

---

## 🔬 Méthodologie Complète Appliquée

**✅ Pipeline sklearn** : Encapsulation preprocessing + modèle (évite data leakage)  
**✅ ColumnTransformer** : Traitement séparé numeric (imputation médiane + scaling) et categorical (imputation mode + one-hot)  
**✅ Cross-Validation K-Fold (K=5)** : 5 splits train/validation indépendants → moyenne robuste  
**✅ Comparaison de 4 algorithmes** : LinearRegression, Random Forest, SVR, LightGBM  
**✅ GridSearchCV** : Optimisation des hyperparamètres du champion (144 combinaisons Energy, 108 CO2)  
**✅ Feature Engineering** : 10 nouvelles features créées (interactions, ratios, encodages)  
**✅ Data Leakage Prevention** : Exclusion rigoureuse des targets croisées + Pipeline  
**✅ Transformation Log** : Normalisation des distributions asymétriques  
**✅ Reproductibilité** : random_state=42 partout, shuffle=True dans KFold

---

### 💡 Enseignements Clés

1. **Les features d'interaction sont CRUCIALES** : `EnergyPerSurface` et `SurfaceGasInteraction` dominent le TOP pour CO2
   - Target 1 (Énergie) : Features structurelles (surfaces) dominantes
   - Target 2 (CO2) : Feature énergétique + interactions dominent → SiteEnergyUse est le prédicteur #1
2. **GridSearchCV + Pipeline + CV = Optimisation robuste** :
   - ❌ GridSearchCV sur Train/Test simple : Risque de suroptimisation
   - ✅ GridSearchCV avec Pipeline + CV : Optimisation robuste, chaque combinaison évaluée sur 5 folds
   - Résultat : +1.8% pour Energy, +0.05% pour CO2
3. **Cross-Validation >> Train/Test simple** :
   - ❌ Train/Test : 1 seul split → dépend du random_state, y_train_pred évalue sur données vues
   - ✅ CV K-Fold : 5 splits → estimation robuste, toujours évalue sur données non vues
4. **Pipeline sklearn évite le data leakage** : Preprocessing (scaling, encoding) appliqué UNIQUEMENT sur train fold à chaque itération
5. **La transformation log est cruciale** pour gérer l'asymétrie des distributions
6. **`SiteEnergyUse` est le prédicteur #1 pour CO2** : 17% d'importance, confirme lien direct énergie → émissions
7. **ThirdLargestPropertyUseType\* conservées** : Malgré 50% NaN, contribuent à la prédiction (conseil mentor validé)

---

### 📊 Évaluation de l'Overfitting

**Qu'est-ce que l'Overfit Gap ?**  
Écart entre R² moyen sur les données d'entraînement (Train) et R² moyen sur validation croisée (CV).

**Interprétation :**

- **Gap = R² Train - R² CV**
- Plus le gap est faible, meilleur est le compromis biais-variance

**Seuils d'interprétation (règle générale) :**

- **0-0.05** : Excellent - Modèle bien régularisé
- **0.05-0.15** : Très bon - Compromis idéal biais-variance
- **0.15-0.25** : Acceptable - Overfitting modéré
- **>0.25** : Problématique - Régularisation nécessaire

**💡 Note importante :** Un gap de 0 est **impossible** et **indésirable** (underfitting). Les modèles complexes ont naturellement un petit gap.

---

### 🚀 Applications Business

**Ces modèles permettent de :**

- Prédire la consommation énergétique (R² = 0.73) et les émissions CO2 (R² = 0.92) de nouveaux bâtiments **avant construction**
- Prioriser les actions d'efficacité énergétique selon les features importantes :
  - **Énergie** : Agir sur la surface des bâtiments, le type d'usage
  - **CO2** : Réduire la consommation énergétique (prédicteur #1), optimiser l'intensité énergétique
- Estimer l'impact de rénovations structurelles (changement de surface, conversion source énergie)
- Identifier les bâtiments à fort potentiel d'amélioration énergétique

---

## 🏆 TABLEAU RÉCAPITULATIF FINAL

| Target      | Meilleur Modèle | R² CV (Base) | R² CV (Optimisé) | Amélioration | Algorithme |
| ----------- | --------------- | ------------ | ---------------- | ------------ | ---------- |
| **Énergie** | Random Forest   | 0.7159       | **0.7288**       | **+1.8%**    | Bagging    |
| **CO2**     | LightGBM        | 0.9214       | **0.9218**       | **+0.05%**   | Boosting   |

**💡 Conclusion :** GridSearchCV avec Pipeline + Cross-Validation a permis d'optimiser les deux modèles, avec une amélioration significative pour Energy (+1.8%) et une confirmation des hyperparamètres optimaux pour CO2 (+0.05%).

---
